## Setting up 

In [1]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, CustomJS, LabelSet
import numpy as np

output_notebook()

Loading BokehJS ...

## Defining Variables

In [2]:
# long string variables
twentysix = [1,2,3,4,5,6,7,8,9,10,11,12,
             13,14,15,16,17,18,19,20,21,22,23,24,25,26]
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m',
            'n','o','p','q','r','s','t','u','v','w','x','y','z']
alphabet = alphabet*2


# aesthetic variables
x_offset = -4.5    # letter annotation offset from circle edge
y_offset = -7    # letter annotation offset from circle edge

seg_color = 'gold'    # segment color
line_width = 3.0    # segment line width
seg_alpha = 0.6    # segment alpha

circ_color = 'gold'    # circle color
circ_size = 23    # circle size
circ_alpha = 0.4    # circle alpha
hov_color = 'gold'    # hover color for circles
hov_alpha = 0.8    # hover alpha for circles

font = 'Garamond'


# some points
x = np.array([50,55])
x = np.repeat(x,26)
y = twentysix*2

## Creating X and Y values for paths

In [3]:
# X values
xvals = [[50,55]]*26

In [4]:
# Y values
# rotor 8
yvals = [[1,6],[2,11],[3,17],[4,8],[5,20],[6,12],[7,24],[8,15],[9,3],
    [10,2],[11,10],[12,19],[13,16],[14,4],[15,26],[16,18],[17,1],
    [18,13],[19,5],[20,23],[21,14],[22,9],[23,21],[24,25],[25,7],
    [26,22]]

# Rotor Links - Rotor 8 [elephant]

In [5]:
# establish links
links = {
    0:[31],     1:[36],      2:[42],     3:[33],     4:[45],     5:[37],      
    6:[49],     7:[40],      8:[28],     9:[27],    10:[35],    11:[44],   
   12:[41],    13:[29],     14:[51],    15:[43],    16:[26],    17:[38],    
   18:[30],    19:[48],     20:[39],    21:[34],    22:[46],    23:[50],    
   24:[32],    25:[47], 
        
   26:[16],    27:[9],      28:[8],     29:[13],    30:[18],    31:[0],
   32:[24],    33:[3],      34:[21],    35:[10],    36:[1],     37:[5],
   38:[17],    39:[20],     40:[7],     41:[12],    42:[2],     43:[15],
   44:[11],    45:[4],      46:[22],    47:[25],    48:[19],    49:[6],  
   50:[23],    51:[14]
}

## Create Graph and Add Features

In [6]:
# create graph
p = figure(width=400, height=800, tools="",
           toolbar_location=None,title='Rotor 8')

# add letter labels
lsource = ColumnDataSource(data=dict(x = x, y = y, names = alphabet))
labels = LabelSet(x='x', y='y', text='names', level='annotation',
                  x_offset=x_offset, y_offset=y_offset, source=lsource,
                  render_mode='canvas', text_font=font, 
                  text_color='black', text_font_style='bold')
p.add_layout(labels)

# add hover glyphs
source = ColumnDataSource({'x0': [], 'y0': [], 'x1': [], 'y1':[]})
sr = p.segment(x0='x0',y0='y0',x1='x1', y1='y1', color=seg_color, 
               alpha=seg_alpha, line_width=line_width, source=source,)
cr = p.circle(x, y, color=circ_color, size=circ_size, alpha=circ_alpha, 
              hover_color=hov_color, hover_alpha=hov_alpha)

# add path glyphs
p.multi_line(xvals,yvals,color='gray',alpha=0.3)

# remove gridlines and axis labels
p.xgrid.visible = False
p.ygrid.visible = False
p.axis.visible = False

# JS for hover tool
code = """
var links = %s;
var data = {'x0': [], 'y0': [], 'x1': [], 'y1': []};
var cdata = circle.get('data');
var indices = cb_data.index['1d'].indices;
for (i=0; i < indices.length; i++) {
    ind0 = indices[i]
    for (j=0; j < links[ind0].length; j++) {
        ind1 = links[ind0][j];
        data['x0'].push(cdata.x[ind0]);
        data['y0'].push(cdata.y[ind0]);
        data['x1'].push(cdata.x[ind1]);
        data['y1'].push(cdata.y[ind1]);
    }
}
segment.set('data', data);
""" % links

# Call JS and Add HoverTool
callback = CustomJS(args={'circle':cr.data_source, 
                          'segment':sr.data_source}, code=code)
p.add_tools(HoverTool(tooltips=None,callback=callback,renderers=[cr]))

# Display Graph

In [7]:
show(p)